In [1]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.10.1
True


In [239]:
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.utils.data import DataLoader

from mmengine.evaluator import BaseMetric
from mmengine.model import BaseModel
from mmengine.runner import Runner

In [241]:
class MMVGG16(BaseModel):
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.vgg16()

    def forward(self, imgs, labels, mode):
        x = self.resnet(imgs)
        if mode == 'loss':
            return {'loss': F.cross_entropy(x, labels)}
        elif mode == 'predict':
            return x, labels

a = MMVGG16()
a

class Accuracy(BaseMetric):
    def process(self, data_batch, data_samples):
        score, gt = data_samples
        self.results.append({
            'batch_size': len(gt),
            'correct': (score.argmax(dim=1) == gt).sum().cpu(),
        })

    def compute_metrics(self, results):
        total_correct = sum(item['correct'] for item in results)
        total_size = sum(item['batch_size'] for item in results)
        return dict(accuracy=100 * total_correct / total_size)


norm_cfg = dict(mean=[124.508, 116.050, 106.438], std=[58.577, 57.310, 57.437])
train_dataloader = DataLoader(batch_size=32,
                              shuffle=True,
                              dataset=torchvision.datasets.CIFAR10(
                                  'data/cifar10',
                                  train=True,
                                  download=True,
                                  transform=transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(**norm_cfg)
                                  ])))

val_dataloader = DataLoader(batch_size=32,
                            shuffle=False,
                            dataset=torchvision.datasets.CIFAR10(
                                'data/cifar10',
                                train=False,
                                download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize(**norm_cfg)
                                ])))

runner = Runner(
    model=MMVGG16(),
    work_dir='./work_dir',
    train_dataloader=train_dataloader,
    optim_wrapper=dict(optimizer=dict(type=SGD, lr=0.001, momentum=0.9)),
    train_cfg=dict(by_epoch=True, max_epochs=5, val_interval=1),
    val_dataloader=val_dataloader,
    val_cfg=dict(),
    val_evaluator=dict(type=Accuracy),
)
runner.train()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified
11/24 00:27:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 2146494974
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA

MMVGG16(
  (data_preprocessor): BaseDataPreprocessor()
  (resnet): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [242]:
%%writefile configs/vgg/vgg19_alzheimer_axial_view.py
_base_ = [
    '../_base_/models/vgg16.py',
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    # '../_base_/schedules/imagenet_bs256.py',
    '../_base_/default_runtime.py'
]

pretrained = 'https://download.openmmlab.com/mmclassification/v0/vgg/vgg19_bn_batch256_imagenet_20210208-da620c4f.pth'

model = dict(
    type='ImageClassifier', 
    
    backbone=dict(
        type='VGG',
        depth=19,
        frozen_stages=-1,
        num_classes=3,
        norm_eval=True,
        # style='pytorch',
        init_cfg = dict(
            type='Pretrained', 
            checkpoint=pretrained,
            prefix='backbone',
        )
    ),
    neck=None,
    head=dict(
        # type='MultiLabelClsHead',
        # loss=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        topk = (1, 3),
        # cal_acc=True,
    ))

dataset_type = 'CustomDataset'
data_preprocessor = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    dict(type='ResizeEdge', scale=256),  
    dict(type='RandomResizedCrop', scale=224),     # Random scaling and cropping
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),         # prepare images and labels
]

test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    dict(type='ResizeEdge', scale=256),  
    dict(type='CenterCrop', crop_size=244),     # center crop
    dict(type='PackInputs'),                 # prepare images and labels
]

train_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/train',
        classes='data/classes.txt',
        ann_file='train_ann.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/validation',
        classes='data/classes.txt',
        ann_file='val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=32,
    num_workers=5,
    dataset=dict(
        type='CustomDataset',
        data_root='data',
        data_prefix='YoriDataset_vgg/test',
        classes='data/classes.txt',
        ann_file='test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))

test_evaluator = val_evaluator 

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=256)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Writing configs/vgg/vgg19_alzheimer_axial_view.py


# Train

In [243]:
!python ./tools/train.py \
    configs/vgg/vgg19_alzheimer_axial_view.py \
    --work-dir work_dirs/alzheimer/axial/vgg16 \
    # --auto-scale-lr

configs/vgg/vgg19_alzheimer_axial_view.py
11/26 01:21:20 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1629880455
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=co

In [4]:
!pwd

/home/user/mmpretrain


In [88]:
!python tools/test.py configs/vgg/vgg16_alzheimer_axial_view.py work_dirs/alzheimer/axial/vgg16/epoch_3.pth

11/14 22:59:17 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 248044611
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_5

In [97]:
!python tools/analysis_tools/confusion_matrix.py \
    configs/vgg/vgg16_alzheimer_axial_view.py \
    work_dirs/alzheimer/axial/vgg16/epoch_100.pth \
    --show

11/14 23:50:33 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 96908555
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50

# Heatmap

In [190]:
import os
import subprocess
train_AD = 'data/YoriDataset_vgg/train/AD'
train_MCI = 'data/YoriDataset_vgg/train/MCI'
train_NC = 'data/YoriDataset_vgg/train/NC'

val_AD = 'data/YoriDataset_vgg/validation/AD'
val_MCI = 'data/YoriDataset_vgg/validation/MCI'
val_NC = 'data/YoriDataset_vgg/validation/NC'

test_AD = 'data/YoriDataset_vgg/test/AD'
test_MCI = 'data/YoriDataset_vgg/test/MCI'
test_NC = 'data/YoriDataset_vgg/test/NC'


trainheat_AD = 'data/vgg_heat/train/AD'
trainheat_MCI = 'data/vgg_heat/train/MCI'
trainheat_NC = 'data/vgg_heat/train/NC'

valheat_AD = 'data/vgg_heat/validation/AD'
valheat_MCI = 'data/vgg_heat/validation/MCI'
valheat_NC = 'data/vgg_heat/validation/NC'

testheat_AD = 'data/vgg_heat/test/AD'
testheat_MCI = 'data/vgg_heat/test/MCI'
testheat_NC = 'data/vgg_heat/test/NC'

count = 0
for i in os.listdir(val_NC):
    count+=1
    temp = os.path.join(val_NC,i)
    out = valheat_NC + '/heat_'+i
    print(out)  
    subprocess.run(["python", "tools/visualization/vis_cam.py", 
                temp,
                 "configs/vgg/vgg16_alzheimer_axial_view.py",
                 "work_dirs/alzheimer/axial/vgg16/epoch_100.pth",
                 # "--target-layers", "backbone.features.16",
                 "--method", "GradCAM",
                 "--save-path", out,
                 "--eigen-smooth", "--aug-smooth"
                 ], text=True)
    print(count)
    # break

data/vgg_heat/validation/NC/heat_53x-133.png


/home/user/miniconda3/envs/open-mmlab/lib/python3.8/site-packages/mmcv/cnn/bricks/conv_module.py:207: UserWarning: Unnecessary conv bias before batch/instance norm
  warnings.warn(


Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
13
Automatically choose the last norm layer "backbone.features.16.bn" as the target layer.
1
data/vgg_heat/validation/NC/heat_11x-119.png


/home/user/miniconda3/envs/open-mmlab/lib/python3.8/site-packages/mmcv/cnn/bricks/conv_module.py:207: UserWarning: Unnecessary conv bias before batch/instance norm
  warnings.warn(


Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
13
Automatically choose the last norm layer "backbone.features.16.bn" as the target layer.
2
data/vgg_heat/validation/NC/heat_60x-131.png


...

Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
13
Automatically choose the last norm layer "backbone.features.16.bn" as the target layer.
88
data/vgg_heat/validation/NC/heat_48x-135.png


/home/user/miniconda3/envs/open-mmlab/lib/python3.8/site-packages/mmcv/cnn/bricks/conv_module.py:207: UserWarning: Unnecessary conv bias before batch/instance norm
  warnings.warn(


Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
13
Automatically choose the last norm layer "backbone.features.16.bn" as the target layer.
89
data/vgg_heat/validation/NC/heat_7x-136.png


/home/user/miniconda3/envs/open-mmlab/lib/python3.8/site-packages/mmcv/cnn/bricks/conv_module.py:207: UserWarning: Unnecessary conv bias before batch/instance norm
  warnings.warn(


Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
13
Automatically choose the last norm layer "backbone.features.16.bn" as the target layer.
90


In [147]:
!python tools/visualization/vis_cam.py \
    data/YoriDataset_vgg/train/MCI/100x-103.png \
    configs/vgg/vgg16_alzheimer_axial_view.py \
    work_dirs/alzheimer/axial/vgg16/epoch_100.pth \
    # --target-layers backbone.features.17 \
    --method GradCAM
    # GradCAM++, XGradCAM, EigenCAM, EigenGradCAM, LayerCAM

Loads checkpoint by local backend from path: work_dirs/alzheimer/axial/vgg16/epoch_100.pth
backbone VGG(
  (features): Sequential(
    (0): ConvModule(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activate): ReLU(inplace=True)
    )
    (1): ConvModule(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activate): ReLU(inplace=True)
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ConvModule(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activate): ReLU(inplace=True)
    )
    (4): ConvModule(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activate): ReLU(inplace=True)
    )
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ConvModule(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activate): Re

## Train Heatmap

In [201]:
%%writefile configs/vgg/vgg16_alzheimer_axial_view_heat.py
_base_ = [
    '../_base_/models/vgg16bn.py',
    '../_base_/datasets/imagenet_bs32_pil_resize.py',
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    # '../_base_/schedules/imagenet_bs256.py',
    '../_base_/default_runtime.py'
]

pretrained = 'https://download.openmmlab.com/mmclassification/v0/vgg/vgg16_bn_batch256_imagenet_20210208-7e55cd29.pth'

model = dict(
    type='ImageClassifier', 
    
    backbone=dict(
        type='VGG',
        depth=16,
        frozen_stages=-1,
        num_classes=3,
        norm_eval=True,
        init_cfg = dict(
            type='Pretrained', 
            checkpoint=pretrained,
            prefix='backbone',
        )
    ),
    neck=None,
    head=dict(
        topk = (1, ),
        # cal_acc=True,
    ))

dataset_type = 'CustomDataset'
data_preprocessor = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=244),  
    
    dict(type='RandomResizedCrop', scale=224),     # Random scaling and cropping
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='Normalize', **data_preprocessor),
    dict(type='PackInputs'),         # prepare images and labels
]

test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    dict(type='ResizeEdge', scale=244),  
    dict(type='CenterCrop', crop_size=244),     # center crop
    dict(type='Normalize', **data_preprocessor),
    dict(type='PackInputs'),                 # prepare images and labels
]

train_dataloader = dict(
    batch_size=16,
    num_workers=1,
    dataset=dict(
        type='CustomDataset',
        data_root='',
        data_prefix='data/vgg_heat/train',
        classes='data/classes.txt',
        # ann_file='data/train_ann.txt',
        pipeline=train_pipeline
    ),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=32,
    num_workers=1,
    dataset=dict(
        type='CustomDataset',
        data_root='',
        data_prefix='data/vgg_heat/validation',
        classes='data/classes.txt',
        # ann_file='data/val_ann.txt',
        pipeline=test_pipeline
    ),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=32,
    num_workers=1,
    dataset=dict(
        type='CustomDataset',
        data_root='',
        data_prefix='data/vgg_heat/test',
        classes='data/classes.txt',
        # ann_file='data/test_ann.txt',
        pipeline=test_pipeline
    ),    
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, ))
test_evaluator = val_evaluator 

# optimizer
# optim_wrapper = dict(
#     optimizer=dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001))

# # learning policy
# param_scheduler = dict(
#     type='MultiStepLR', by_epoch=True, milestones=[30, 60, 90], gamma=0.1)

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=128)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Overwriting configs/vgg/vgg16_alzheimer_axial_view_heat.py


In [202]:
!python ./tools/train.py \
    configs/vgg/vgg16_alzheimer_axial_view_heat.py \
    --work-dir work_dirs/alzheimer/axial/vgg16_heat \
    --auto-scale-lr

configs/vgg/vgg16_alzheimer_axial_view_heat.py
11/16 17:20:13 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 41050527
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch